# TDVP numerical test \#1

The goal is to use the TDVP to simulate the time evolution of a simple Hamiltonian, when state space is restricted to some manifold Furthermore a goal is to compare this method with directly solving the SE and projecting to the manifold.

The first example will be $\mathcal{M}= S^2$ and $H=\sigma_x$ to generate an X-rotation of a 1-qubit system. Note that the Bloch-sphere is a restriction of the entire 2-dim $\mathcal{H}$ onto a sphere. 

In [3]:
from math import * 
import numpy as np
import scipy as sp

import jax
import jax.numpy as jnp

import matplotlib.pyplot as plt

import qutip as qt

In [4]:
H = np.matrix(qt.sigmax().full())
H

matrix([[0.+0.j, 1.+0.j],
        [1.+0.j, 0.+0.j]])

In [87]:
def psi(params:np.array):
    theta, phi = params
    ket0=np.matrix(qt.basis(2,0).full().flatten())
    ket1=np.matrix(qt.basis(2,1).full().flatten())
    # assert -np.pi<=theta<np.pi, f"theta={theta} out of bounds."
    # assert 0<=phi<2*np.pi, f"phi={phi} out of bounds."
    return np.cos(theta/2)*ket0 + np.exp(1j*phi)*np.sin(theta/2)*ket1

In [6]:
qt.basis(2,0).full().flatten()

array([1.+0.j, 0.+0.j])

In [7]:
psi([pi/2,0])

array([0.70710678+0.j, 0.70710678+0.j])

In [8]:
jacpsi = jax.jacfwd(psi)

In [9]:
z=np.array([1.1,2.2])

In [10]:
# jacpsi(z)

In [11]:
def f(x,y):
    return np.array([x**2+y, 5*+np.sin(y)]) 

In [12]:
# jf = jax.jacfwd(f)
# X=np.array([1.0,1.0])
# jf(X)

In [13]:
def finitediff(f,i=0,epsilon=0.00000001):
    def df(params):
        theta,phi=params
        if i==0:
            dtheta=theta+epsilon
            dphi=phi
        if i==1:
            dphi=phi+epsilon
            dtheta=theta
        return (f([dtheta,dphi])-f([theta,phi]))/epsilon
    return df

In [43]:
def df(func,epsilon=0.00000001):
    def d_func(params):
        return(np.matrix([finitediff(func,i=0,epsilon=epsilon)(params),finitediff(func,i=1,epsilon=epsilon)(params)]))
    return d_func

In [61]:
df(psi)((0,0))

matrix([[0. +0.j, 0.5+0.j],
        [0. +0.j, 0. +0.j]])

In [63]:
dpsi = finitediff(psi,1)

In [64]:
dpsi([1,1])

array([ 0.        +0.j        , -0.40342268+0.25903472j])

In [16]:
def G(params,psi)->np.array:
    def G_j_k(j, k):
        dpsi_j = finitediff(psi, j)
        dpsi_k = finitediff(psi, k)
        return np.dot(dpsi_j(params), dpsi_k(params))
    return(np.matrix([[G_j_k(0, 0), G_j_k(0, 1)],
                     [G_j_k(1, 0), G_j_k(1, 1)]]))


In [17]:
G([1,1],psi)[0]

matrix([[-0.02266177+0.17507411j, -0.19128685-0.08754387j]])

In [83]:
def time_evo(psi,params,stepsize=0.1):
    params = params
    #compute partial derivatives
    dpsi = df(psi)
    # define step from t to t+\Delta t
    def step(params):
        # define gradient dphi H phi
        grad = np.matrix([(dpsi(params)[i]*H*psi(params).T).item(0)for i in range(2)])
        # calculate step vector and add
        step_vec = -1j*stepsize*np.linalg.inv(G(params,psi))*grad.T
        step_lenght = np.linalg.norm(step_vec)
        return (step_lenght,params + step_vec)
    t=0
    step_lenght = 1

    while t<=10000 and step_lenght>10^(-10):
        t+=1
        step_lenght, params = step(params)

    return step(params)


    

In [89]:
time_evo(psi,(1,1))

ValueError: matrix must be 2-dimensional

In [39]:
np.matrix([[1,2],[3,4]])

TypeError: 'list' object cannot be interpreted as an integer

In [59]:
(np.matrix([1,1])*H*np.matrix([1,1]).T).shape

(1, 1)